In [5]:
from ctypes import WINFUNCTYPE, byref, c_ubyte, create_unicode_buffer, windll
from ctypes.wintypes import BOOL, HWND, LPARAM, RECT

import cv2
import numpy as np
from unidecode import unidecode


class Window_Manager:
    def __init__(self, window_names=None):
        self.window_names = window_names or ["PokeMMO", "RokeMMO"]
        self.window_name = None
        self.EnumWindowsProc = WINFUNCTYPE(BOOL, HWND, LPARAM)
        self.GetWindowText = windll.user32.GetWindowTextW
        self.GetWindowTextLength = windll.user32.GetWindowTextLengthW
        self.IsWindowVisible = windll.user32.IsWindowVisible
        self.GetDC = windll.user32.GetDC
        self.CreateCompatibleDC = windll.gdi32.CreateCompatibleDC
        self.GetClientRect = windll.user32.GetClientRect
        self.CreateCompatibleBitmap = windll.gdi32.CreateCompatibleBitmap
        self.SelectObject = windll.gdi32.SelectObject
        self.BitBlt = windll.gdi32.BitBlt
        self.SRCCOPY = 0x00CC0020
        self.GetBitmapBits = windll.gdi32.GetBitmapBits
        self.DeleteObject = windll.gdi32.DeleteObject
        self.ReleaseDC = windll.user32.ReleaseDC
        self.handle = None

    def get_window_name(self):
        """Get the window name of the PokeMMO game."""
        callback = self.EnumWindowsProc(self.foreach_window)

        windll.user32.EnumWindows(callback, 0)

        if self.window_name is None:
            raise Exception("Failed to find window name for PokeMMO")

        self.handle = windll.user32.FindWindowW(None, self.window_name)

        print(f"window name: {self.window_name}")
        return self.window_name

    def foreach_window(self, hwnd, lParam):
        if self.IsWindowVisible(hwnd):
            length = self.GetWindowTextLength(hwnd)
            buff = create_unicode_buffer(length + 1)
            self.GetWindowText(hwnd, buff, length + 1)
            ascii_text = unidecode(buff.value)
            if ascii_text in self.window_names:
                self.window_name = buff.value
                return True
        return True

    def get_current_img_BRG(self):
        """Capture a screenshot of the PokeMMO game."""
        self.GetClientRect = windll.user32.GetClientRect
        self.GetDC = windll.user32.GetDC
        self.CreateCompatibleDC = windll.gdi32.CreateCompatibleDC
        self.CreateCompatibleBitmap = windll.gdi32.CreateCompatibleBitmap
        self.SelectObject = windll.gdi32.SelectObject
        self.BitBlt = windll.gdi32.BitBlt
        self.GetBitmapBits = windll.gdi32.GetBitmapBits
        self.DeleteObject = windll.gdi32.DeleteObject
        self.ReleaseDC = windll.user32.ReleaseDC
        self.SRCCOPY = 0xCC0020

        r = RECT()
        self.GetClientRect(self.handle, byref(r))
        width, height = r.right, r.bottom
        dc = self.GetDC(self.handle)
        cdc = self.CreateCompatibleDC(dc)
        bitmap = self.CreateCompatibleBitmap(dc, width, height)
        self.SelectObject(cdc, bitmap)
        self.BitBlt(cdc, 0, 0, width, height, dc, 0, 0, self.SRCCOPY)
        total_bytes = width * height * 4
        buffer = bytearray(total_bytes)
        byte_array = c_ubyte * total_bytes
        self.GetBitmapBits(bitmap, total_bytes, byte_array.from_buffer(buffer))
        self.DeleteObject(bitmap)
        self.DeleteObject(cdc)
        self.ReleaseDC(self.handle, dc)
        image_normal = np.frombuffer(buffer, dtype=np.uint8).reshape(height, width, 4)
        img_BRG = cv2.cvtColor(image_normal, cv2.COLOR_BGRA2BGR)
        return img_BRG  # return img_BRG


In [6]:
wm=Window_Manager()

In [7]:
import ctypes

# The array of bytes to search for
aob = bytes([0x41, 0x0F, 0xBE, 0x71, 0x10, 0x45])

# The process ID of the game
pid = 1234  # Replace with your process ID

# Open the process
PROCESS_ALL_ACCESS = (0x000F0000 | 0x00100000 | 0xFFF)
process_handle = ctypes.windll.kernel32.OpenProcess(PROCESS_ALL_ACCESS, False, pid)

# The range of addresses to search
start_address = 0x00000000
end_address = 0x7FFFFFFF

# The size of the memory to read at once
chunk_size = 4096

# The buffer to read the memory into
buffer = (ctypes.c_ubyte * chunk_size)()

# The number of bytes read
bytes_read = ctypes.c_ulong(0)

# Loop over the memory
for address in range(start_address, end_address, chunk_size):
    # Read the memory
    ctypes.windll.kernel32.ReadProcessMemory(process_handle, address, buffer, chunk_size, ctypes.byref(bytes_read))

    # Search for the array of bytes
    for i in range(chunk_size - len(aob)):
        if bytes(buffer[i:i+len(aob)]) == aob:
            print("Found array of bytes at address: 0x%X" % (address + i))

# Close the process handle
ctypes.windll.kernel32.CloseHandle(process_handle)


KeyboardInterrupt: 

In [ ]:
import ctypes
import os
import sys
from ctypes import wintypes

# Define necessary WinAPI structures and constants
PROCESS_ALL_ACCESS = 0x1F0FFF
MEM_COMMIT = 0x1000
PAGE_EXECUTE_READWRITE = 0x40

# Define WinAPI functions
OpenProcess = ctypes.windll.kernel32.OpenProcess
OpenProcess.argtypes = [wintypes.DWORD, wintypes.BOOL, wintypes.DWORD]
OpenProcess.restype = wintypes.HANDLE

VirtualAllocEx = ctypes.windll.kernel32.VirtualAllocEx
VirtualAllocEx.argtypes = [wintypes.HANDLE, wintypes.LPVOID, ctypes.c_size_t, wintypes.DWORD, wintypes.DWORD]
VirtualAllocEx.restype = wintypes.LPVOID

ReadProcessMemory = ctypes.windll.kernel32.ReadProcessMemory
ReadProcessMemory.argtypes = [wintypes.HANDLE, wintypes.LPVOID, wintypes.LPVOID, ctypes.c_size_t, ctypes.POINTER(ctypes.c_size_t)]
ReadProcessMemory.restype = wintypes.BOOL

WriteProcessMemory = ctypes.windll.kernel32.WriteProcessMemory
WriteProcessMemory.argtypes = [wintypes.HANDLE, wintypes.LPVOID, wintypes.LPCVOID, ctypes.c_size_t, ctypes.POINTER(ctypes.c_size_t)]
WriteProcessMemory.restype = wintypes.BOOL

CloseHandle = ctypes.windll.kernel32.CloseHandle
CloseHandle.argtypes = [wintypes.HANDLE]
CloseHandle.restype = wintypes.BOOL

# Obtain PID
pid = os.getpid() # replace with appropriate function to get the PID

# Open the process
h_process = OpenProcess(PROCESS_ALL_ACCESS, False, pid)
if not h_process:
    print("Failed to open process")
    sys.exit(1)

# Allocate memory
addr = VirtualAllocEx(h_process, None, 2, MEM_COMMIT, PAGE_EXECUTE_READWRITE)
if not addr:
    print("Failed to allocate memory")
    sys.exit(1)

# Read the original bytes
old = (ctypes.c_byte * 8)()
bytes_read = ctypes.c_size_t(0)
if not ReadProcessMemory(h_process, addr, old, 8, ctypes.byref(bytes_read)):
    print("Failed to read original bytes")
    sys.exit(1)

# Write new bytes
new_bytes = (ctypes.c_byte * 8)(0xE9, 0, 0, 0, 0, 0x0F, 0x1F, 0x00)
bytes_written = ctypes.c_size_t(0)
if not WriteProcessMemory(h_process, addr, new_bytes, 8, ctypes.byref(bytes_written)):
    print("Failed to write new bytes")
    sys.exit(1)

# Don't forget to close the handle
CloseHandle(h_process)


In [ ]:
import ctypes

def read_memory(address):
    # Cast the address to a pointer
    ptr = ctypes.cast(address, ctypes.POINTER(ctypes.c_int))

    # Dereference the pointer
    return ptr.contents.value

address = 0xF16A0FD4  # the address you want to read
print(read_memory(address))


: 

: 